In [33]:
import boto3
import sagemaker
from sagemaker import get_execution_role
# from sagemaker.image_uris import retrieve
from sagemaker.s3 import *
import sys

if int(sagemaker.__version__.split('.')[0]) == 2:
    !{sys.executable} -m pip install sagemaker==1.72.0
    print("Installing previous SageMaker Version. Please restart the kernel")
else:
    print("Version is good")

role = get_execution_role()
sess = sagemaker.Session(default_bucket=None)
region = boto3.session.Session().region_name
print("Region = {}".format(region))
sm = boto3.Session().client('sagemaker')


Version is good
Region = eu-central-1


In [34]:

# see https://sagemaker.readthedocs.io/en/stable/api/utility/image_uris.html for which inputs to use
# see https://github.com/aws/deep-learning-containers/blob/master/available_images.md for registry paths with custom algorithms


training_image = "763104351884.dkr.ecr.us-east-1.amazonaws.com/pytorch-training:1.6.0-gpu-py36-cu101-ubuntu16.04"
print (training_image)

763104351884.dkr.ecr.us-east-1.amazonaws.com/pytorch-training:1.6.0-gpu-py36-cu101-ubuntu16.04


In [35]:
!pip install sagemaker-experiments 
from sagemaker.analytics import ExperimentAnalytics
from smexperiments.experiment import Experiment
from smexperiments.trial import Trial
from smexperiments.trial_component import TrialComponent
from smexperiments.tracker import Tracker

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/pytorch_p36/bin/python -m pip install --upgrade pip' command.


In [36]:
rawbucket= sess.default_bucket() # Alternatively you can use our custom bucket here. 
data_bucket = "treetracker-training-images"
prefix = 'sagemaker-modelmonitor' # use this prefix to store all files pertaining to this workshop.

dataprefix = prefix + '/data'
traindataprefix = prefix + '/train_data'
testdataprefix = prefix + '/test_data'
testdatanolabelprefix = prefix + '/test_data_no_label'
trainheaderprefix = prefix + '/train_headers'

s3_train_key = "imnet"
s3_validation_key = ""
s3_train = 's3://{}/{}/'.format(data_bucket, s3_train_key)
s3_validation = 's3://{}/{}/'.format(data_bucket, s3_validation_key)
sagemaker_train = 's3://{}/{}/'.format(rawbucket, s3_train_key)
sagemaker_validation = 's3://{}/{}/'.format(rawbucket, s3_validation_key)


In [37]:
train_data_location = rawbucket + '/' + traindataprefix
test_data_location = rawbucket+'/'+testdataprefix
print("Training data location = {}".format(train_data_location))
print("Test data location = {}".format(test_data_location))

Training data location = sagemaker-eu-central-1-053061259712/sagemaker-modelmonitor/train_data
Test data location = sagemaker-eu-central-1-053061259712/sagemaker-modelmonitor/test_data


In [38]:

from sagemaker.processing import ProcessingInput, ProcessingOutput

pre_input = ProcessingInput(source=s3_train, destination=None, input_name="pre")
pre_output = ProcessingOutput(source=None, destination=sagemaker_train)



In [39]:
from sagemaker.pytorch import PyTorch

estimator = PyTorch(entry_point='mobilenet_v2.py',
                    role=role,
                    framework_version='1.4.0',
                    train_instance_count=2,
                    train_instance_type='ml.c4.xlarge',
                    hyperparameters={
                        'epochs': 6,
                        'backend': 'gloo'
                    })

In [40]:
estimator.fit({"training": s3_train})

INFO:sagemaker:Creating training-job with name: pytorch-training-2020-12-11-03-48-49-783


2020-12-11 03:48:50 Starting - Starting the training job...
2020-12-11 03:48:52 Starting - Launching requested ML instances......
2020-12-11 03:50:12 Starting - Preparing the instances for training.........
2020-12-11 03:51:23 Downloading - Downloading input data.................................
2020-12-11 03:57:22 Training - Training image download completed. Training in progress..bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2020-12-11 03:57:23,233 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
2020-12-11 03:57:23,236 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2020-12-11 03:57:23,246 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2020-12-11 03:57:24,360 sagemaker-containers INFO     Imported frame

UnexpectedStatusException: Error for Training job pytorch-training-2020-12-11-03-48-49-783: Failed. Reason: AlgorithmError: ExecuteUserScriptError:
Command "/opt/conda/bin/python mobilenet_v2.py --backend gloo --epochs 6"
Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to /root/.cache/torch/checkpoints/mobilenet_v2-b0353104.pth
  0%|          | 0.00/13.6M [00:00<?, ?B/s] 73%|ââââââââ  | 9.88M/13.6M [00:00<00:00, 103MB/s]100%|ââââââââââ| 13.6M/13.6M [00:00<00:00, 102MB/s]
Traceback (most recent call last):
  File "mobilenet_v2.py", line 548, in <module>
    trainer.train(parser.parse_args())
  File "mobilenet_v2.py", line 315, in train
    self.batch_size = args.batch_size
AttributeError: 'tuple' object has no attribute 'batch_size'